# Prerequisites

- Host OS: Ubuntu 20.04 lts
- Using Docker Image 'mltooling/ml-workspace-gpu' (docker pull mltooling/ml-workspace-gpu)
- Single Nvidia GPU (RTX 3080)

# 0. Import libraries

In [1]:
import bert_ensemble_functions
from datasets import load_dataset

import transformers
import datasets
import huggingface_hub
import pyarrow
import torch
print(transformers.__version__)
print(datasets.__version__)
print(huggingface_hub.__version__)
print(pyarrow.__version__)
print(torch.__version__)

4.22.1
2.4.0
0.9.1
9.0.0
1.9.0+cu111


In [2]:
# 'You can use tf32' if you are acessing Ampere hardware
# torch.backends.cuda.matmul.allow_tf32 = True

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    device_count = torch.cuda.device_count()
    print("device_count: {}".format(device_count))
    for device_num in range(device_count):
        print("device {} capability {}".format(
            device_num,
            torch.cuda.get_device_capability(device_num)))
        print("device {} name {}".format(
            device_num, 
            torch.cuda.get_device_name(device_num)))
else:
    device = torch.device("cpu")
    print("no cuda device")

device_count: 1
device 0 capability (8, 6)
device 0 name NVIDIA GeForce RTX 3080


In [3]:
#### The number of CPU cores
!grep -c processor /proc/cpuinfo

20


In [4]:
#### GPU information
!nvidia-smi

Tue Nov 22 01:17:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   48C    P8    31W / 370W |    253MiB / 12288MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 1. Customize Train Strategy

In [5]:
num_cpus = 16
num_gpus = 1
seed = 1234
model_name = "xlm-roberta-base" # bert-base-multilingual-cased, klue/roberta-base, bert-base-cased, etc.

text_column = 'treatment_effect'
label_column = 'label'
id_column = 'id'
custom_dir = f"sev_{text_column}_ensemble"

train_proportion = 0.5 # train set : eval set = 5 : 5
do_hpo = True

# If you want to search best hyperparameters using ray tune, parameters below should be set
n_trials = 5
std = 0.1
patience = 5

# 2. Import Data

2 files are needed (`{data_name}_train.csv` and `{data_name}_test.csv`) in your data directory (in this case, `data_splited/`).

In [6]:
data_name = "cardiovascular_sev_dataset" 

dataset = load_dataset('csv', data_files={'train': f'../data_split/{data_name}_train.csv',
                                          'test': f'../data_split/{data_name}_test.csv'})
dataset

Using custom data configuration default-9aa18915b5f32f1a
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-9aa18915b5f32f1a/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'past_history', 'treatment_effect', 'examination', 'label'],
        num_rows: 3756
    })
    test: Dataset({
        features: ['id', 'past_history', 'treatment_effect', 'examination', 'label'],
        num_rows: 940
    })
})

# 3. Data Preprocessing

In [7]:
train_dataset, eval_dataset, test_dataset = bert_ensemble_functions.preprocessing(dataset = dataset,
                                                                                  text_column = text_column, 
                                                                                  label_column = label_column,
                                                                                  id_column = id_column,
                                                                                  model_name = model_name,
                                                                                  train_proportion = train_proportion,
                                                                                  seed = seed,
                                                                                  custom_tokenizer_dir = custom_dir)

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-9aa18915b5f32f1a/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-db8c8a49ac2c2b04.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-9aa18915b5f32f1a/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-c4935f191b182aaf.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-9aa18915b5f32f1a/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-e4bb750fe7f81549.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-9aa18915b5f32f1a/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-a7390e752f32feb6.arrow


Removing rows with missing value...
Done. (1/4)
Removing special characters...
Done. (2/4)
Tokenining the text column...


Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-9aa18915b5f32f1a/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-c84994ca965f96aa.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-9aa18915b5f32f1a/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-39e157ba98991116.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/csv/default-9aa18915b5f32f1a/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-c87756cc8d41612b.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/csv/default-9aa18915b5f32f1a/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-c87756cc8d41612b.arrow


Done. (3/4)
Spliting train-evaluation-test set...
Done. (4/4)


# 4. Modeling

In [8]:
trainer = bert_ensemble_functions.modeling(train_dataset=train_dataset,
                                           eval_dataset=eval_dataset,
                                           model_name='xlm-roberta-base',
                                           num_gpus=num_gpus,
                                           num_cpus=num_cpus,
                                           seed=seed,
                                           output_dir='./output',
                                           logging_dir="./logs",
                                           do_hpo=do_hpo,
                                           std = std,
                                           n_trials = n_trials,
                                           patience = patience,
                                           hpo_result_dir = "./hpo-results",
                                           hpo_result_dir_subfolder_name = 'tune_transformer_pbt'
                                          )

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f548f32366559214515ec137cdd16002968bf6/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f

== Status ==
Current time: 2022-11-22 01:17:42 (running for 00:00:00.17)
Memory usage on this node: 9.8/31.1 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 16.0/16 CPUs, 1.0/1 GPUs, 0.0/14.32 GiB heap, 0.0/7.16 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/syc/BERT_XGB_ensemble_classification_binary/hpo-results/tune_transformer_pbt
Number of trials: 5/5 (4 PENDING, 1 RUNNING)
+------------------------+----------+--------------------+-----------+-------------+----------------+--------------+
| Trial name             | status   | loc                |   w_decay |          lr |   train_bs/gpu |   num_epochs |
|------------------------+----------+--------------------+-----------+-------------+----------------+--------------|
| _objective_76597_00000 | RUNNING  | 172.17.0.3:3933377 |  0.186633 | 2.75091e-05 |              8 |           17 |
| _objective_76597_00001 | PENDING  |                    |  0.287442 | 4.50373e-05 |              8 |  

== Status ==
Current time: 2022-11-22 01:18:14 (running for 00:00:32.27)
Memory usage on this node: 14.4/31.1 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 16.0/16 CPUs, 1.0/1 GPUs, 0.0/14.32 GiB heap, 0.0/7.16 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/syc/BERT_XGB_ensemble_classification_binary/hpo-results/tune_transformer_pbt
Number of trials: 5/5 (4 PENDING, 1 RUNNING)
+------------------------+----------+--------------------+-----------+-------------+----------------+--------------+
| Trial name             | status   | loc                |   w_decay |          lr |   train_bs/gpu |   num_epochs |
|------------------------+----------+--------------------+-----------+-------------+----------------+--------------|
| _objective_76597_00000 | RUNNING  | 172.17.0.3:3933377 |  0.186633 | 2.75091e-05 |              8 |           17 |
| _objective_76597_00001 | PENDING  |                    |  0.287442 | 4.50373e-05 |              8 | 

== Status ==
Current time: 2022-11-22 01:18:45 (running for 00:01:02.85)
Memory usage on this node: 14.8/31.1 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 16.0/16 CPUs, 1.0/1 GPUs, 0.0/14.32 GiB heap, 0.0/7.16 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/syc/BERT_XGB_ensemble_classification_binary/hpo-results/tune_transformer_pbt
Number of trials: 5/5 (1 PAUSED, 3 PENDING, 1 RUNNING)
+------------------------+----------+--------------------+-----------+-------------+----------------+--------------+-----------+-----------------+------------------+-------------+---------+----------------------+
| Trial name             | status   | loc                |   w_decay |          lr |   train_bs/gpu |   num_epochs |   eval_f1 |   eval_accuracy |   eval_objective |   eval_loss |   epoch |   training_iteration |
|------------------------+----------+--------------------+-----------+-------------+----------------+--------------+-----------+------

== Status ==
Current time: 2022-11-22 01:19:05 (running for 00:01:22.86)
Memory usage on this node: 14.8/31.1 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 16.0/16 CPUs, 1.0/1 GPUs, 0.0/14.32 GiB heap, 0.0/7.16 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/syc/BERT_XGB_ensemble_classification_binary/hpo-results/tune_transformer_pbt
Number of trials: 5/5 (1 PAUSED, 3 PENDING, 1 RUNNING)
+------------------------+----------+--------------------+-----------+-------------+----------------+--------------+-----------+-----------------+------------------+-------------+---------+----------------------+
| Trial name             | status   | loc                |   w_decay |          lr |   train_bs/gpu |   num_epochs |   eval_f1 |   eval_accuracy |   eval_objective |   eval_loss |   epoch |   training_iteration |
|------------------------+----------+--------------------+-----------+-------------+----------------+--------------+-----------+------

...

== Status ==
Current time: 2022-11-22 01:36:49 (running for 00:19:07.10)
Memory usage on this node: 15.7/31.1 GiB
PopulationBasedTraining: 5 checkpoints, 1 perturbs
Resources requested: 16.0/16 CPUs, 1.0/1 GPUs, 0.0/14.32 GiB heap, 0.0/7.16 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/syc/BERT_XGB_ensemble_classification_binary/hpo-results/tune_transformer_pbt
Number of trials: 5/5 (4 PAUSED, 1 RUNNING)
+------------------------+----------+--------------------+-----------+-------------+----------------+--------------+-----------+-----------------+------------------+-------------+---------+----------------------+
| Trial name             | status   | loc                |   w_decay |          lr |   train_bs/gpu |   num_epochs |   eval_f1 |   eval_accuracy |   eval_objective |   eval_loss |   epoch |   training_iteration |
|------------------------+----------+--------------------+-----------+-------------+----------------+--------------+-----------+-----------------

== Status ==
Current time: 2022-11-22 01:37:11 (running for 00:19:29.49)
Memory usage on this node: 15.8/31.1 GiB
PopulationBasedTraining: 6 checkpoints, 1 perturbs
Resources requested: 16.0/16 CPUs, 1.0/1 GPUs, 0.0/14.32 GiB heap, 0.0/7.16 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/syc/BERT_XGB_ensemble_classification_binary/hpo-results/tune_transformer_pbt
Number of trials: 5/5 (4 PAUSED, 1 RUNNING)
+------------------------+----------+--------------------+-----------+-------------+----------------+--------------+-----------+-----------------+------------------+-------------+---------+----------------------+
| Trial name             | status   | loc                |   w_decay |          lr |   train_bs/gpu |   num_epochs |   eval_f1 |   eval_accuracy |   eval_objective |   eval_loss |   epoch |   training_iteration |
|------------------------+----------+--------------------+-----------+-------------+----------------+--------------+-----------+-----------------

== Status ==
Current time: 2022-11-22 01:37:31 (running for 00:19:49.50)
Memory usage on this node: 15.8/31.1 GiB
PopulationBasedTraining: 6 checkpoints, 1 perturbs
Resources requested: 16.0/16 CPUs, 1.0/1 GPUs, 0.0/14.32 GiB heap, 0.0/7.16 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/syc/BERT_XGB_ensemble_classification_binary/hpo-results/tune_transformer_pbt
Number of trials: 5/5 (4 PAUSED, 1 RUNNING)
+------------------------+----------+--------------------+-----------+-------------+----------------+--------------+-----------+-----------------+------------------+-------------+---------+----------------------+
| Trial name             | status   | loc                |   w_decay |          lr |   train_bs/gpu |   num_epochs |   eval_f1 |   eval_accuracy |   eval_objective |   eval_loss |   epoch |   training_iteration |
|------------------------+----------+--------------------+-----------+-------------+----------------+--------------+-----------+-----------------

Result for _objective_76597_00003:
  date: 2022-11-22_01-37-50
  done: true
  episodes_total: 0
  epoch: 0.85
  eval_accuracy: 0.7177848775292864
  eval_f1: 0.0
  eval_loss: 0.690936803817749
  eval_objective: 0.7177848775292864
  eval_runtime: 11.7296
  eval_samples_per_second: 160.108
  eval_steps_per_second: 20.035
  experiment_id: c9c65d4d04684178a90399cc85101cb7
  hostname: 3481a8a2ae33
  iterations_since_restore: 1
  node_ip: 172.17.0.3
  objective: 1.4355697550585729
  pid: 3933377
  time_since_restore: 53.59272289276123
  time_this_iter_s: 53.59272289276123
  time_total_s: 194.792884349823
  timestamp: 1669081070
  timesteps_since_restore: 0
  timesteps_total: 0
  training_iteration: 1
  trial_id: '76597_00003'
  warmup_time: 0.0016245841979980469
  
== Status ==
Current time: 2022-11-22 01:37:50 (running for 00:20:08.05)
Memory usage on this node: 15.8/31.1 GiB
PopulationBasedTraining: 6 checkpoints, 1 perturbs
Resources requested: 0/16 CPUs, 0/1 GPUs, 0.0/14.32 GiB heap, 0.0/

2022-11-22 01:37:50,411	INFO tune.py:758 -- Total run time: 1208.37 seconds (1208.04 seconds for the tuning loop).


In [9]:
trainer.train()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f548f32366559214515ec137cdd16002968bf6/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f

Step,Training Loss,Validation Loss,Accuracy,F1,Objective
50,No log,0.688789,0.724175,0.268362,0.992536
100,No log,0.667161,0.739084,0.219745,0.958829
150,No log,0.406630,0.884984,0.781818,1.666802
200,No log,0.313866,0.898296,0.828700,1.726996
250,No log,0.365102,0.854633,0.782123,1.636755
300,No log,0.335697,0.920660,0.859566,1.780227
350,No log,0.344983,0.926518,0.873394,1.799912
400,No log,0.289818,0.930777,0.876894,1.807671
450,No log,0.329968,0.920660,0.868258,1.788918
500,0.384800,0.374922,0.922258,0.867031,1.789289


The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1878
  Batch size = 8
/opt/conda/lib/python3.8/site-packages/transformers/trainer.py:1808: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  nn.utils.clip_grad_norm_(
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignor

TrainOutput(global_step=754, training_loss=0.28987899446360943, metrics={'train_runtime': 701.721, 'train_samples_per_second': 34.792, 'train_steps_per_second': 1.075, 'total_flos': 6417804862341120.0, 'train_loss': 0.28987899446360943, 'epoch': 12.99})

In [10]:
# save the pretrained model
trainer.model.save_pretrained(custom_dir)

Configuration saved in sev_treatment_effect_ensemble/config.json
Model weights saved in sev_treatment_effect_ensemble/pytorch_model.bin


In [11]:
df = bert_ensemble_functions.evaluation(trainer = trainer,
                                        eval_dataset = eval_dataset,
                                        text_column_name = text_column
                                        )
df

The following columns in the test set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1878
  Batch size = 8


,id,text,treatment_effect_pred_0,treatment_effect_pred_1,label,pred
0,17370,chest painNSTEMI CKMBTnT 18268147HTNDMTTE 1831...,3.906250,-4.230469,0,0
1,18264,STEMICAOD 1vd sp PTCA c stent insertion at dLC...,3.353516,-3.634766,1,0
2,15281,HD중 내원 1시간 전 시작된 chest pain 으로 STEMI imp 하에 본원...,2.365234,-2.642578,0,0
3,51130,Problem list1 strokeRt MCA infarction 2018년 2...,-1.747070,1.633789,1,1
4,46160,상기 77세 남환 DM CKD old CVA pelvic fr 201808 NSTE...,3.744141,-4.074219,0,0
...,...,...,...,...,...,...
1873,38025,Nausea dizziness 주소로 ER 내원 감별하게 시행한 CAOD CT상 i...,2.957031,-3.332031,0,0
1874,9988,CAOD 2vdCAD2VD PTCA c stent at pmLAD Resolute...,3.458984,-3.884766,0,0
1875,40275,chest pain Current Diagnosis NSTEMI sp PTCA c...,3.865234,-4.191406,0,0
1876,49338,chest discomfortCurrent diagnosis recent MICAO...,3.261719,-3.580078,0,0


In [12]:
# Check the classification result of each XLM-RoBERTa Model 
from sklearn.metrics import (
    confusion_matrix, 
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score
)

print("Confusion Matrix")
print(confusion_matrix(df.label, df.pred))
print("-------------------------")
accuracy = accuracy_score(df.label, df.pred)
f1 = f1_score(df.label, df.pred)
recall = recall_score(df.label, df.pred)
precision = precision_score(df.label, df.pred)
print(f"Accuracy: {accuracy}")
print(f"F1 score: {f1}")
print(f"Recall: {recall}")
print(f"Precision: {precision}")

Confusion Matrix
[[1256   92]
 [  54  476]]
-------------------------
Accuracy: 0.9222577209797657
F1 score: 0.8670309653916212
Recall: 0.8981132075471698
Precision: 0.8380281690140845


In [13]:
df.to_csv(f'./{text_column}_bert_result_df.csv', index=False)